In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# This needs to be defined by the drop down in the UI
subgroup1 = "woman"
subgroup2 = "man"
subgroup3 = "non-binary"

In [3]:
data = load_dataset("c4", "en", split= "train", streaming = True)

In [4]:
grab_n = 10000
# For streaming data
print('Note: Just taking the first %s instances.' % grab_n)
data_head = data.take(grab_n)
#data_head = [["there is a woman with a hairbrush"],["there is a woman with a hairbrush"],["there is a woman with a hairbrush"],["there is a man with a dog"],["there is a man with a dog"]]
df = pd.DataFrame(data_head, columns=["text"])
# If not streaming, use:
#df = pd.json_normalize(data)

Note: Just taking the first 10000 instances.


In [5]:
def count_vocab_frequencies(df):
    """
    Based on an input pandas DataFrame with a 'text' column, 
    this function will count the occurrences of all words
    with a frequency higher than 'cutoff' and will return another DataFrame
    with the rows corresponding to the different vocabulary words
    and the column to the count count of that word.
    """
    # Move this up as a constant in larger code.
    batch_size = 10
    
    # We do this to calculate per-word statistics
    df['text'] = df['text'].str.lower()
    # Regex for pulling out single words
    cvec = CountVectorizer(token_pattern=u"(?u)\\b\\w+\\b", lowercase=True)
    
    # We also do this because we need to have the tokenization per sentence 
    # so that we can look at co-occurrences of words across sentences for nPMI calculation
    sent_tokenizer = cvec.build_tokenizer()
    df['tokenized'] = df.text.apply(sent_tokenizer)
    
    # Fast calculation of single word counts
    cvec.fit(df.text)
    document_matrix = cvec.transform(df.text)
    batches = np.linspace(0, df.shape[0], batch_size).astype(int)
    i = 0
    tf = []
    while i < len(batches) - 1:
        batch_result = np.sum(document_matrix[batches[i]:batches[i+1]].toarray(), axis=0)
        tf.append(batch_result)
        i += 1
    term_freq_df = pd.DataFrame([np.sum(tf, axis=0)], columns=cvec.get_feature_names()).transpose()
    
    # Now organize everything into the dataframes
    term_freq_df.columns = ['count']
    term_freq_df.index.name = 'word'
    sorted_term_freq_df = pd.DataFrame(term_freq_df.sort_values(by='count', ascending=False)['count'])
    return sorted_term_freq_df, df

In [6]:
term_df, df = count_vocab_frequencies(df)
# p(word).  Note that multiple occurrences of a word in a sentence increases its probability.
# We may want to do something about that.
term_df['proportion'] = term_df['count']/float(sum(term_df['count']))
# Sanity check
print(term_df.head())
print(term_df.tail())

       count  proportion
word                    
the   186019    0.050628
and   107893    0.029365
to    103090    0.028058
of     89417    0.024336
a      81307    0.022129
             count    proportion
word                            
interestel       1  2.721674e-07
interethnic      1  2.721674e-07
interfaced       1  2.721674e-07
interfacing      1  2.721674e-07
𐌼𐌿𐌽𐌳𐍃            1  2.721674e-07


In [7]:
def get_PMI(df_coo, subgroup):
    # PMI(x;y) = h(y) - h(y|x)
    #          = h(subgroup) - h(subgroup|word)
    #          = log (p(subgroup|word) / p(subgroup))
    # nPMI additionally divides by -log(p(x,y)) = -log(p(x|y)p(y))
    #
    # Calculation of p(subgroup)
    subgroup_prob = term_df.loc[subgroup]['proportion']
    # Apply a function to all words to calculate log p(subgroup|word)
    # The word is indexed by mlb.classes_ ; 
    # we pull out the word using the mlb.classes_ index and then get its count using our main term_df
    # Calculation:
    # p(subgroup|word) = count(subgroup,word) / count(word)
    #                  = x.values             / term_df.loc[mlb.classes_[x.index]]['count']
    pmi_df = pd.DataFrame(df_coo.apply(lambda x: np.log(x.values/term_df.loc[mlb.classes_[x.index]]['count']/subgroup_prob)))
    pmi_df.columns = ['pmi']
    # If all went well, this will be correlated with high frequency words
    # Until normalizing
    # Note: A potentially faster solution for adding count, npmi, can be based on this:
    # #df_test['size_kb'],  df_test['size_mb'], df_test['size_gb'] = zip(*df_test['size'].apply(sizes))
    return pmi_df

In [8]:
def get_nPMI(pmi_df, df_coo):
    normalize_df = pd.DataFrame(df_coo.apply(lambda x: -np.log(x.values/term_df.loc[mlb.classes_[x.index]]['count'] * term_df.loc[mlb.classes_[x.index]]['proportion'])))
    # npmi_df = pmi_df/normalize_df
    npmi_df = pd.DataFrame(pmi_df['pmi']/normalize_df[0])
    npmi_df.columns = ['npmi']
    return npmi_df

In [9]:
def get_count(df_coo, subgroup):
    # TBH I have no clue why this works.
    count_df = pd.DataFrame(df_coo.apply(lambda x: pd.Series(x.values, mlb.classes_[x.index])))
    count_df.columns=['count']
    return count_df

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

# Makes a sparse vector (shape: # sentences x # words),
# with the count of each word per sentence.
mlb = MultiLabelBinarizer()
df_mlb = pd.DataFrame(mlb.fit_transform(df['tokenized']))

# Calculates PMI metrics
paired_results = pd.DataFrame()
results_dict = {}
for subgroup in (subgroup1, subgroup2):
    # Index of the subgroup word in the sparse vector
    subgroup_idx = np.where(mlb.classes_ == subgroup)[0][0]
    # Dataframe for the subgroup (with counts)
    df_subgroup = df_mlb.iloc[:, subgroup_idx]
    # Create cooccurence matrix for the given subgroup and all other words.
    # Note it also includes the word itself, so that count should maybe be subtracted 
    # (the word will always co-occur with itself)
    print('Calculating co-occurrences')
    df_coo = pd.DataFrame(df_mlb.T.dot(df_subgroup))#.drop(index=subgroup_idx, axis=1)
    print('Getting counts for subgroup...')
    count_df = get_count(df_coo, subgroup)
    print(count_df)
    print('Calculating PMI...')
    pmi_df = get_PMI(df_coo, subgroup)
    print(pmi_df)
    print('Calculating nPMI...')
    #pmi_df_pair[subgroup] = pmi_df
    npmi_df = get_nPMI(pmi_df, df_coo)
    print(npmi_df)
    #results_df = pd.concat([count_df,pmi_df,npmi_df], axis=1)
    paired_results[subgroup + '-pmi']  = pmi_df['pmi']
    paired_results[subgroup + '-npmi'] = npmi_df['npmi']
    paired_results[subgroup + '-count'] = count_df['count'] 

Calculating co-occurrences
Getting counts for subgroup...
            count
0               3
00              4
000            27
0000            0
000002          0
...           ...
ﬁnancial        0
ﬁrst            0
ﬂexibility      0
ﬂoors           0
𐌼𐌿𐌽𐌳𐍃           0

[100533 rows x 1 columns]
Calculating PMI...
                 pmi
word                
0           3.456918
00          4.686165
000         5.617594
0000            -inf
000002          -inf
...              ...
ﬁnancial        -inf
ﬁrst            -inf
ﬂexibility      -inf
ﬂoors           -inf
𐌼𐌿𐌽𐌳𐍃           -inf

[100533 rows x 1 columns]
Calculating nPMI...


/Users/margaretmitchell/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/margaretmitchell/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


                npmi
word                
0           0.246601
00          0.341295
000         0.475221
0000             NaN
000002           NaN
...              ...
ﬁnancial         NaN
ﬁrst             NaN
ﬂexibility       NaN
ﬂoors            NaN
𐌼𐌿𐌽𐌳𐍃            NaN

[100533 rows x 1 columns]
Calculating co-occurrences
Getting counts for subgroup...
            count
0              22
00              8
000            59
0000            0
000002          0
...           ...
ﬁnancial        0
ﬁrst            0
ﬂexibility      0
ﬂoors           0
𐌼𐌿𐌽𐌳𐍃           0

[100533 rows x 1 columns]
Calculating PMI...
                 pmi
word                
0           4.369043
00          4.299007
000         5.318989
0000            -inf
000002          -inf
...              ...
ﬁnancial        -inf
ﬁrst            -inf
ﬂexibility      -inf
ﬂoors           -inf
𐌼𐌿𐌽𐌳𐍃           -inf

[100533 rows x 1 columns]
Calculating nPMI...


/Users/margaretmitchell/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


                npmi
word                
0           0.363306
00          0.329744
000         0.481823
0000             NaN
000002           NaN
...              ...
ﬁnancial         NaN
ﬁrst             NaN
ﬂexibility       NaN
ﬂoors            NaN
𐌼𐌿𐌽𐌳𐍃            NaN

[100533 rows x 1 columns]


/Users/margaretmitchell/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
print(paired_results.dropna())

      woman-pmi  woman-npmi  woman-count   man-pmi  man-npmi  man-count
word                                                                   
0      3.456918    0.246601            3  4.369043  0.363306         22
00     4.686165    0.341295            4  4.299007  0.329744          8
000    5.617594    0.475221           27  5.318989  0.481823         59
00am   6.725219    0.444882            1  5.644914  0.373419          1
00pm   5.769707    0.381674            1  4.689402  0.310210          1
...         ...         ...          ...       ...       ...        ...
іn     7.641509    0.505496            1  6.561204  0.434033          1
іt     7.353827    0.486466            1  6.273522  0.415002          1
ү᧐u    9.433269    0.624024            1  8.352964  0.552560          1
һow    9.433269    0.624024            1  8.352964  0.552560          1
ꮇy     9.433269    0.624024            1  8.352964  0.552560          1

[13826 rows x 6 columns]


In [12]:
# woman - man: If it's negative, it's man-biased; if it's positive, it's woman positive.
npmi_bias = paired_results[subgroup1 + '-npmi'] - paired_results[subgroup2 + '-npmi'] #pd.DataFrame(results_dict[subgroup1]['npmi'] - results_dict[subgroup2]['npmi']).dropna()
paired_results['npmi_bias'] = npmi_bias.dropna()
paired_results = paired_results.dropna()
#pmi_bias = pd.DataFrame(pmi_df_pair[subgroup1] - pmi_df_pair[subgroup2])

In [13]:
n = 20

In [14]:
print("@%s, the %s bias is:\t%.2f" % (n, subgroup2, np.abs(sum(paired_results.npmi_bias[:n].values))))
print("@%s, the %s bias is:\t%.2f" % (n, subgroup1, sum(paired_results.npmi_bias[-n:].values)))

@20, the man bias is:	0.39
@20, the woman bias is:	1.32


In [15]:
print("Top %s most %s-biased words" % (n, subgroup2))
paired_results.npmi_bias.sort_values(ascending=True)[:n]

Top 20 most man-biased words


word
man         -0.260967
pure        -0.248211
foot        -0.243202
failed      -0.224456
league      -0.211324
decade      -0.205872
minds       -0.204651
squad       -0.204100
leg         -0.199634
precisely   -0.190910
route       -0.190002
gained      -0.189576
origin      -0.186229
desires     -0.183848
premier     -0.183546
blocked     -0.181568
wave        -0.181540
actor       -0.179628
versions    -0.179389
thinks      -0.178664
Name: npmi_bias, dtype: float64

In [16]:
print("Top %s most %s-biased words" % (n,subgroup1))
paired_results.npmi_bias.sort_values(ascending=True)[-n:].sort_values(ascending=False)

Top 20 most woman-biased words


word
woman             0.296502
quit              0.252845
swedish           0.238161
complement        0.233675
vitamins          0.232567
representation    0.229159
miraculously      0.223465
activism          0.219417
coaster           0.219417
shoots            0.215766
childbirth        0.215424
grooming          0.215424
murray            0.212285
strengthening     0.210629
mai               0.209576
alice             0.209439
malaysia          0.206596
workplace         0.205919
flights           0.202572
helpless          0.201522
Name: npmi_bias, dtype: float64

In [17]:
print(paired_results.info())

<class 'pandas.core.frame.DataFrame'>
Index: 13826 entries, 0 to ꮇy
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   woman-pmi    13826 non-null  float64
 1   woman-npmi   13826 non-null  float64
 2   woman-count  13826 non-null  int64  
 3   man-pmi      13826 non-null  float64
 4   man-npmi     13826 non-null  float64
 5   man-count    13826 non-null  int64  
 6   npmi_bias    13826 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 864.1+ KB
None


In [18]:
print(paired_results.head())

      woman-pmi  woman-npmi  woman-count   man-pmi  man-npmi  man-count  \
word                                                                      
0      3.456918    0.246601            3  4.369043  0.363306         22   
00     4.686165    0.341295            4  4.299007  0.329744          8   
000    5.617594    0.475221           27  5.318989  0.481823         59   
00am   6.725219    0.444882            1  5.644914  0.373419          1   
00pm   5.769707    0.381674            1  4.689402  0.310210          1   

      npmi_bias  
word             
0     -0.116704  
00     0.011551  
000   -0.006601  
00am   0.071464  
00pm   0.071464  
